In [1]:
import  os
import  tensorflow as tf
import  numpy as np
from    tensorflow import keras
from    tensorflow.keras import datasets, layers, optimizers, models
from    tensorflow.keras import regularizers
from utils import LoadFishDataUtil
from tensorflow.keras.utils import plot_model


## loading train data and generate the data config

In [2]:
data_dir ='/media/xingbo/Storage/fish_identification/data/SESSION_TENT/SESSION1'
BATCH_SIZE = 8
IMG_SIZE=160
IMG_WIDTH=320
IMG_HEIGHT=60

In [ ]:
myloadData = LoadFishDataUtil(data_dir,BATCH_SIZE,IMG_WIDTH,IMG_HEIGHT)
train_dataset,val_dataset,test_dataset,STEPS_PER_EPOCH, CLASS_NAMES,class_num = myloadData.loadFishData()


## define the model 

In [ ]:
class VGG16(keras.Model):


    def __init__(self, input_shape,num_classes):
        """

        :param input_shape: [32, 32, 3]
        """
        super(VGG16, self).__init__()

        weight_decay = 0.000
        self.num_classes = num_classes

        model = models.Sequential()

        model.add(layers.Conv2D(64, (3, 3), padding='same',
                         input_shape=input_shape, kernel_regularizer=regularizers.l2(weight_decay)))
        model.add(layers.Activation('relu'))
        model.add(layers.BatchNormalization())
        model.add(layers.Dropout(0.3))

        model.add(layers.Conv2D(64, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
        model.add(layers.Activation('relu'))
        model.add(layers.BatchNormalization())

        model.add(layers.MaxPooling2D(pool_size=(2, 2)))

        model.add(layers.Conv2D(128, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
        model.add(layers.Activation('relu'))
        model.add(layers.BatchNormalization())
        model.add(layers.Dropout(0.4))

        model.add(layers.Conv2D(128, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
        model.add(layers.Activation('relu'))
        model.add(layers.BatchNormalization())

        model.add(layers.MaxPooling2D(pool_size=(2, 2)))

        model.add(layers.Conv2D(256, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
        model.add(layers.Activation('relu'))
        model.add(layers.BatchNormalization())
        model.add(layers.Dropout(0.4))

        model.add(layers.Conv2D(256, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
        model.add(layers.Activation('relu'))
        model.add(layers.BatchNormalization())
        model.add(layers.Dropout(0.4))

        model.add(layers.Conv2D(256, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
        model.add(layers.Activation('relu'))
        model.add(layers.BatchNormalization())

        model.add(layers.MaxPooling2D(pool_size=(2, 2)))


        model.add(layers.Conv2D(512, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
        model.add(layers.Activation('relu'))
        model.add(layers.BatchNormalization())
        model.add(layers.Dropout(0.4))

        model.add(layers.Conv2D(512, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
        model.add(layers.Activation('relu'))
        model.add(layers.BatchNormalization())
        model.add(layers.Dropout(0.4))

        model.add(layers.Conv2D(512, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
        model.add(layers.Activation('relu'))
        model.add(layers.BatchNormalization())

        model.add(layers.MaxPooling2D(pool_size=(2, 2)))


        model.add(layers.Conv2D(512, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
        model.add(layers.Activation('relu'))
        model.add(layers.BatchNormalization())
        model.add(layers.Dropout(0.4))

        model.add(layers.Conv2D(512, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
        model.add(layers.Activation('relu'))
        model.add(layers.BatchNormalization())
        model.add(layers.Dropout(0.4))

        model.add(layers.Conv2D(512, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
        model.add(layers.Activation('relu'))
        model.add(layers.BatchNormalization())

        model.add(layers.MaxPooling2D(pool_size=(2, 2)))
        model.add(layers.Dropout(0.5))

        model.add(layers.Flatten())
        model.add(layers.Dense(512,kernel_regularizer=regularizers.l2(weight_decay)))
        model.add(layers.Activation('relu'))
        model.add(layers.BatchNormalization())

        model.add(layers.Dropout(0.5))
        model.add(layers.Dense(self.num_classes))
        # model.add(layers.Activation('softmax'))


        self.model = model


    def call(self, x):

        x = self.model(x)

        return x

## loading model

In [ ]:
num_classes = class_num
batch_size = 32
epochs = 10
validation_steps = 20
# build model and optimizer
model = VGG16([IMG_WIDTH,IMG_HEIGHT, 3],class_num)
model.compile(optimizer=keras.optimizers.Adam(0.001),
              loss=keras.losses.CategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])
model.build(input_shape=(None, IMG_WIDTH,IMG_HEIGHT, 3))

model.load_weights('model/vggfish20191121TENT.h5')

In [ ]:
#plot_model(model)     # 画出模型结构图，并保存成图片,to_file = 'a simple convnet.png'
print(model.summary())                                 # 打印模型概况


In [ ]:
#【1】创建一个新model, 使得它的输出(outputs)是 VGG19 中任意层的输出(output)
model2 = models.Model(inputs=model.input, outputs=model.get_layer('block4_pool').output)
print(model2.summary())                                 # 打印模型概况

In [ ]:
test_data_dir ='/home/xingbo/Desktop/fish_identification/data/SESSION_TENT/SESSION1'
#test_data_dir ='/home/xingbo/Desktop/fish_identification/data/SESSION_AQUARIUM/SESSION_test'
BATCH_SIZE = 8
IMG_SIZE=160
SPLIT_WEIGHTS = (0, 0, 1) # Data prepare split wight train vs validation vs test
testloadData = LoadFishDataUtil(test_data_dir,BATCH_SIZE,IMG_SIZE,CLASS_NAMES,SPLIT_WEIGHTS)
sess2_train_dataset,sess2_val_dataset,sess2_test_dataset,sess2_STEPS_PER_EPOCH, sess2_CLASS_NAMES,sess2_class_num = testloadData.loadFishData()

#sess2_test_dataset,sess2_class_num = testloadData.loadTestFishData()
loss0,accuracy0 = model.evaluate(sess2_test_dataset, steps = 20)


In [ ]:

test_data_dir ='/home/xingbo/Desktop/fish_identification/data/SESSION_TANT/SESSION1'
#test_data_dir ='/home/xingbo/Desktop/fish_identification/data/SESSION_AQUARIUM/SESSION_test'
BATCH_SIZE = 8
IMG_SIZE=160
testloadData = LoadFishDataUtil(test_data_dir,BATCH_SIZE,IMG_SIZE,CLASS_NAMES)
sess2_test_dataset,sess2_class_num = testloadData.loadTestFishData()
loss0,accuracy0 = model.evaluate(sess2_test_dataset, steps = 20)


In [ ]:
from tensorflow.keras.models import Sequential
from    tensorflow.keras import datasets, layers, optimizers, models
from    tensorflow.keras import regularizers

model = models.Sequential() # 顺序模型

# 输入层
model.add(layers.Dense(7, input_shape=(4,)))  # Dense就是常用的全连接层
model.add(layers.Activation('sigmoid')) # 激活函数

# 隐层
model.add(layers.Dense(13))  # Dense就是常用的全连接层
model.add(layers.Activation('sigmoid')) # 激活函数

# 输出层
model.add(layers.Dense(5))
model.add(layers.Activation('softmax'))

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=["accuracy"])

model.summary()


In [ ]:
from tensorflow.keras.models import Sequential
from    tensorflow.keras import datasets, layers, optimizers, models
from    tensorflow.keras import regularizers
model = models.Sequential() # 顺序模型
# input: 100x100 images with 3 channels -> (100, 100, 3) tensors.
# this applies 32 convolution filters of size 3x3 each.
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(100, 100, 3)))
model.add(layers.Conv2D(32, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D(pool_size=(2, 2)))
model.add(layers.Dropout(0.25))

model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D(pool_size=(2, 2)))
model.add(layers.Dropout(0.25))

model.add(layers.Flatten())
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(10, activation='softmax'))

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=["accuracy"])

model.summary()
